# Setup Environment and MedSam2

In [1]:
# clone the repo
!git clone https://github.com/MedicineToken/Medical-SAM2.git

Cloning into 'Medical-SAM2'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 191 (delta 43), reused 36 (delta 36), pack-reused 136 (from 1)
Receiving objects: 100% (191/191), 2.20 MiB | 5.66 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [2]:
%%writefile /content/Medical-SAM2/setup.py

# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
import os

from setuptools import find_packages, setup

# Package metadata
NAME = "SAM 2"
VERSION = "1.0"
DESCRIPTION = "SAM 2: Segment Anything in Images and Videos"
URL = "https://github.com/facebookresearch/segment-anything-2"
AUTHOR = "Meta AI"
AUTHOR_EMAIL = "segment-anything@meta.com"
LICENSE = "Apache 2.0"

# Read the contents of README file
with open("README.md", "r", encoding="utf-8") as f:
    LONG_DESCRIPTION = f.read()

# Required dependencies
REQUIRED_PACKAGES = [
    "torch>=2.3.1",
    "torchvision>=0.18.1",
    "numpy>=1.24.4",
    "tqdm>=4.66.1",
    "hydra-core>=1.3.2",
    "iopath>=0.1.10",
    "pillow>=9.4.0",
]

EXTRA_PACKAGES = {
    "demo": ["matplotlib>=3.9.1", "jupyter>=1.0.0", "opencv-python>=4.7.0"],
    "dev": ["black==24.2.0", "usort==1.0.2", "ufmt==2.0.0b2"],
}

# By default, we also build the SAM 2 CUDA extension.
# You may turn off CUDA build with `export SAM2_BUILD_CUDA=0`.
BUILD_CUDA = os.getenv("SAM2_BUILD_CUDA", "1") == "1"
# By default, we allow SAM 2 installation to proceed even with build errors.
# You may force stopping on errors with `export SAM2_BUILD_ALLOW_ERRORS=0`.
BUILD_ALLOW_ERRORS = os.getenv("SAM2_BUILD_ALLOW_ERRORS", "1") == "1"

# Catch and skip errors during extension building and print a warning message
# (note that this message only shows up under verbose build mode
# "pip install -v -e ." or "python setup.py build_ext -v")
CUDA_ERROR_MSG = (
    "{}\n\n"
    "Failed to build the SAM 2 CUDA extension due to the error above. "
    "You can still use SAM 2 and it's OK to ignore the error above, although some "
    "post-processing functionality may be limited (which doesn't affect the results in most cases; "
    "(see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).\n"
)


def get_extensions():
    if not BUILD_CUDA:
        return []

    try:
        from torch.utils.cpp_extension import CUDAExtension

        srcs = ["sam2_train/csrc/connected_components.cu"]
        compile_args = {
            "cxx": [],
            "nvcc": [
                "-DCUDA_HAS_FP16=1",
                "-D__CUDA_NO_HALF_OPERATORS__",
                "-D__CUDA_NO_HALF_CONVERSIONS__",
                "-D__CUDA_NO_HALF2_OPERATORS__",
            ],
        }
        ext_modules = [CUDAExtension("sam2_train._C", srcs, extra_compile_args=compile_args)]
    except Exception as e:
        if BUILD_ALLOW_ERRORS:
            print(CUDA_ERROR_MSG.format(e))
            ext_modules = []
        else:
            raise e

    return ext_modules


try:
    from torch.utils.cpp_extension import BuildExtension

    class BuildExtensionIgnoreErrors(BuildExtension):

        def finalize_options(self):
            try:
                super().finalize_options()
            except Exception as e:
                print(CUDA_ERROR_MSG.format(e))
                self.extensions = []

        def build_extensions(self):
            try:
                super().build_extensions()
            except Exception as e:
                print(CUDA_ERROR_MSG.format(e))
                self.extensions = []

        def get_ext_filename(self, ext_name):
            try:
                return super().get_ext_filename(ext_name)
            except Exception as e:
                print(CUDA_ERROR_MSG.format(e))
                self.extensions = []
                return "_C.so"

    cmdclass = {
        "build_ext": (
            BuildExtensionIgnoreErrors.with_options(no_python_abi_suffix=True)
            if BUILD_ALLOW_ERRORS
            else BuildExtension.with_options(no_python_abi_suffix=True)
        )
    }
except Exception as e:
    cmdclass = {}
    if BUILD_ALLOW_ERRORS:
        print(CUDA_ERROR_MSG.format(e))
    else:
        raise e


# Setup configuration
setup(
    name=NAME,
    version=VERSION,
    description=DESCRIPTION,
    long_description=LONG_DESCRIPTION,
    long_description_content_type="text/markdown",
    url=URL,
    author=AUTHOR,
    author_email=AUTHOR_EMAIL,
    license=LICENSE,
    packages=find_packages(exclude="notebooks"),
    package_data={"": ["*.yaml"]},  # SAM 2 configuration files
    include_package_data=True,
    install_requires=REQUIRED_PACKAGES,
    extras_require=EXTRA_PACKAGES,
    python_requires=">=3.10.0",
    ext_modules=get_extensions(),
    cmdclass=cmdclass,
)

Writing /content/Medical-SAM2/setup.py


In [3]:
# move to repo dir
%cd /content/Medical-SAM2

# build setup
!python setup.py build_ext --inplace

/content/Medical-SAM2
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:495: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:414: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:424: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.2
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'sam2_train._C' extension
creating build
creating build/temp.linux-x86_6

In [4]:
# Ensure you're in the correct environment, preferably with Python 3.10 using conda
# conda create --name myenv python=3.10
# conda activate myenv

# Install PyTorch and CUDA
!pip install torch==2.0.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install other dependencies
!pip install tensorboardx matplotlib tqdm pandas nibabel acvl-utils==0.2 antlr4-python3-runtime==4.9.3 anyio==4.4.0 \
argon2-cffi==23.1.0 argon2-cffi-bindings==21.2.0 argparse==1.4.0 arrow==1.3.0 async-lru==2.0.4 asyncio-atexit==1.0.1 \
attrs==23.2.0 babel==2.15.0 backoff==2.2.1 batchgenerators==0.25 batchgeneratorsv2==0.2 beautifulsoup4==4.12.3 bleach==6.1.0 \
certifi==2024.7.4 cffi==1.16.0 charset-normalizer==3.3.2 connected-components-3d==3.18.0 contourpy==1.2.1 cycler==0.12.1 \
defusedxml==0.7.1 deprecated==1.2.14 dicom2nifti==2.4.11 dynamic-network-architectures==0.3.1 einops==0.8.0 fastjsonschema==2.20.0 \
fft-conv-pytorch==1.2.0 filelock==3.15.4 fonttools==4.53.1 fqdn==1.5.1 fsspec==2024.6.1 future==1.0.0 googleapis-common-protos==1.63.2 \
h11==0.14.0 httpcore==1.0.5 httpx==0.27.0 hydra-core==1.3.2 idna==3.7 imagecodecs==2024.6.1 imageio==2.34.2 importlib-metadata==6.11.0 \
iopath==0.1.10 ipympl==0.9.4 ipywidgets==8.1.3 jinja2==3.1.4 joblib==1.4.2 jsonschema==4.23.0 jupyterlab==4.2.4 kiwisolver==1.4.5

# If you want to install Jupyter Lab related packages
!pip install jupyterlab jupyterlab-widgets jupyter-server

# For additional dependencies
!pip install numpy==1.24.4 scikit-learn==1.5.1 monai==1.3.2 scipy==1.14.0 simpleitk==2.3.1

Looking in indexes: https://download.pytorch.org/whl/cu121
ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.1.0+cu121, 2.1.1+cu121, 2.1.2+cu121, 2.2.0+cu121, 2.2.1+cu121, 2.2.2+cu121, 2.3.0+cu121, 2.3.1+cu121, 2.4.0+cu121, 2.4.1+cu121)
ERROR: No matching distribution found for torch==2.0.1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: simpleitk
    Found existing installation: SimpleITK 2.4.0
    Uninstalling SimpleITK-2.4.0:
      Successfully uninstalled SimpleITK-2.4.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: scikit-learn
    Found existing insta

In [5]:
# python version
import sys
print("Python version:", sys.version)

# os info
!lsb_release -a

# gpu version
!nvcc --version

Python version: 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
# be sure it is working and list our packages
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.34.2
acvl_utils                       0.2
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.14
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
antlr4-python3-runtime           4.9.3
anyio                            4.4.0
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arrow                            1.3.0
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.3
astropy-iers-data                0.2024.9.11.18.16.39
astunparse                       1.6.3
async-lru                

# Get Data and Checkpoints

In [7]:
%cd /content/Medical-SAM2

/content/Medical-SAM2


In [8]:
# download all checkpoints
!bash checkpoints/download_ckpts.sh

--2024-09-14 09:03:43--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155906050 (149M) [application/vnd.snesdev-page-table]
Saving to: ‘sam2_hiera_tiny.pt’

sam2_hiera_tiny.pt  100%[===================>] 148.68M   152MB/s    in 1.0s    

2024-09-14 09:03:45 (152 MB/s) - ‘sam2_hiera_tiny.pt’ saved [155906050/155906050]

--2024-09-14 09:03:45--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184309650 (176M) [appl

In [9]:
# get 2d data and unzip
!wget https://huggingface.co/datasets/jiayuanz3/btcv/resolve/main/btcv.zip
!unzip btcv.zip

Streaming output truncated to the last 5000 lines.
  inflating: btcv/Training/image/img0035/10.jpg  
  inflating: btcv/Training/image/img0035/38.jpg  
  inflating: btcv/Training/image/img0035/21.jpg  
  inflating: btcv/Training/image/img0035/35.jpg  
  inflating: btcv/Training/image/img0035/34.jpg  
  inflating: btcv/Training/image/img0035/20.jpg  
  inflating: btcv/Training/image/img0035/36.jpg  
  inflating: btcv/Training/image/img0035/22.jpg  
  inflating: btcv/Training/image/img0035/23.jpg  
  inflating: btcv/Training/image/img0035/37.jpg  
  inflating: btcv/Training/image/img0035/33.jpg  
  inflating: btcv/Training/image/img0035/27.jpg  
  inflating: btcv/Training/image/img0035/26.jpg  
  inflating: btcv/Training/image/img0035/32.jpg  
  inflating: btcv/Training/image/img0035/18.jpg  
  inflating: btcv/Training/image/img0035/24.jpg  
  inflating: btcv/Training/image/img0035/30.jpg  
  inflating: btcv/Training/image/img0035/31.jpg  
  inflating: btcv/Training/image/img0035/25.jpg  

In [10]:
# move downloaded models to checkpoints
!mv /content/Medical-SAM2/sam2_hiera_base_plus.pt /content/Medical-SAM2/sam2_hiera_large.pt /content/Medical-SAM2/sam2_hiera_small.pt /content/Medical-SAM2/sam2_hiera_tiny.pt ./checkpoints/

In [11]:
import os

# create data folder and move refuge dataset
os.makedirs("./data", exist_ok=True)
!mv btcv ./data/

# Training & Validation on 2D Preprocessed Data

In [12]:
# overwrite configuration
# change epoch from 100 to 1 - to test quickly
# can be changed for experimental purposes

In [13]:
%%writefile /content/Medical-SAM2/conf/global_settings.py
""" configurations for this project

author Yunli
"""
import os
from datetime import datetime

CHECKPOINT_PATH = 'checkpoint'

#total training epoches
EPOCH = 1
step_size = 10
i = 1
MILESTONES = []
while i * 5 <= EPOCH:
    MILESTONES.append(i* step_size)
    i += 1

#initial learning rate
#INIT_LR = 0.1

#time of we run the script
TIME_NOW = datetime.now().strftime("%F_%H-%M-%S.%f")

#tensorboard log dir
LOG_DIR = 'runs'

#save weights file per SAVE_EPOCH epoch
SAVE_EPOCH = 10

Overwriting /content/Medical-SAM2/conf/global_settings.py


In [15]:
# using T4 -> OOM
# run command in conda env and validate example preprocessed 3d data
!python train_3d.py -net sam2 -exp_name BTCV_MedSAM2 -sam_ckpt ./checkpoints/sam2_hiera_tiny.pt -sam_config sam2_hiera_t -image_size 1024 -val_freq 1 -prompt bbox -prompt_freq 2 -dataset btcv -data_path ./data/btcv -b 1

US1_J2KR.dcm:   0% 38.0/154k [00:00<01:23, 1.83kB/s]
MR-SIEMENS-DICOM-WithOverlays.dcm:   0% 125/511k [00:00<02:32, 3.34kB/s]
OBXXXX1A.dcm:   0% 119/486k [00:00<02:35, 3.13kB/s]
US1_UNCR.dcm:   0% 226/923k [00:00<02:30, 6.13kB/s]
color3d_jpeg_baseline.dcm:   0% 1.50k/6.14M [00:00<04:42, 21.8kB/s]
2024-09-14 09:05:14.904838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 09:05:15.177788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 09:05:15.252619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 09:05:15.687196: I tensorflow/core/platform/cpu_feature_